# Amazon SageMaker のモデルバイアスモニタリング: SageMaker Clarify による公平性および説明可能性

Amazon SageMaker Clarify は、バイアスが存在する場合にそのバイアスを検出して、モデルが予測に至る経緯を説明するのに役立つ、機械学習モデルを改善するためのサービスです。SageMaker Clarify の公平性および説明可能性の機能は、AWS のお客様にとって、機械学習モデルの信頼性とわかりやすさを高める 1 歩となります。本製品には以下のタスクの支援ツールが搭載されています。

- 機械学習ライフサイクル (データ収集、モデルトレーニング、モデルチューニング、推論向けにデプロイされた機械学習モデルのモニタリング) の各ステージで発生し得るバイアスを計測する
- リスクとコンプライアンスチームならびに外部規制当局に向けたモデルのガバナンスレポートを生成する
- 予測評価に使用されるデータ、モデル、モニタリングについて説明を提供する

## インポートと設定

必要な Python モジュールをインポートするため以下のコードを実行します。

In [ ]:
import os
import copy
import json
import random
import time
import datetime
import boto3
import pandas as pd

from datetime import datetime, timedelta

from sagemaker import get_execution_role, image_uris, Session
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.predictor import Predictor

from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
)

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)

後ほど使用する modelArtifctBucket の名前を決めるため以下のコードを実行します。

In [ ]:
bucket = ''
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'modelartifactbucket' in buckets.name:
        bucket = buckets.name

print(f"Bucket Name: {bucket}")

後ほど使用する SageMaker プロダクションエンドポイント の名前を事前入力するため以下のコードを実行します。

In [ ]:
# Get production endpoint name
endpoint_name = boto3.Session().client('sagemaker').list_endpoints(SortBy='CreationTime')['Endpoints'][0]['EndpointName']

# Get production model name
model_name = boto3.Session().client('sagemaker').list_models(SortBy='CreationTime')['Models'][0]['ModelName']

# Check if the endpoint is in service
client = boto3.client('sagemaker')
result = False
while result is False:
    response = client.describe_endpoint(EndpointName=endpoint_name)
    if (response['EndpointStatus'] == 'InService'):
        print(f"Production Endpoint Name:{endpoint_name}")
        print(f"Production Model Name:{model_name}")
        result = True
    else:
        print("Waiting for endpoint to be active")
        time.sleep(5)
        result = False

以下のコードを実行し、必要な IAM ロール と AWS リージョンを取得します。

In [ ]:
role = get_execution_role()
print(f"RoleArn: {role}")

sagemaker_session = Session()
sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

region = sagemaker_session.boto_region_name
print(f"AWS region: {region}")

以下のコードを実行し、キャプチャしたデータとレポートを保存する Amazon S3 のパスを作成します。

In [ ]:
prefix = 'sagemaker'
s3_key = f"s3://{bucket}/{prefix}"
print(f"S3 key: {s3_key}")

code_prefix = '{}/code'.format(prefix)
data_capture_prefix = f'{prefix}/datacapture'
s3_capture_upload_path = f'{s3_key}/datacapture'
s3_report_path = f'{s3_key}/reports'

print(f"Capture path: {s3_capture_upload_path}")
print(f"Report path: {s3_report_path}")

以下のコードを実行し、トレーニング、テスト、検証用のデータを読み込みます。

In [ ]:
test_dataset = "data/validate.csv"
train_dataset = "data/train.csv"
dataset_type = "text/csv"

with open(train_dataset) as f:
    headers_line = f.readline().rstrip()
all_headers = headers_line.split(",")
label_header = all_headers[0]

## サンプルトラフィックでエンドポイントをテストする

検証用データセットから2つのサンプルサブセットを抽出し、ローカルのcsvファイルに書き出します。以下の2つのファイルが作成されます：

- 特徴量のデータのみを含んだ data-test.csv 
- 特徴量とラベルのデータを含んだ data-test-label.csv

In [ ]:
# Here we are randomly picking subset of data from test datasets.
import itertools

shape = pd.read_csv(test_dataset)

a = [10*i for i in range(3)]
b = [10+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data = shape.drop(shape.columns[[0]],axis=1)
test_data = test_data.iloc[indices]
test_data_with_label = shape.iloc[indices]

In [ ]:
test_data.to_csv("data/data-test.csv",index=False,header=False)
test_data_with_label.to_csv("data/data-test-label.csv",index=False,header=False)

以下のコードを実行し、推論を実行します。前のステップで取得したサンプルデータセットを使用します。実行後、予測結果を "0" または "1" で取得できます。

In [ ]:
# Invoke Endpoint with test data for a quick test
print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")
predictions = ''

with open('data/data-test.csv', 'r') as f:
    for row in f:
        print(".", end="", flush=True)
        payload = row.rstrip('\n')
        response = sagemaker_runtime_client.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType="text/csv",
                                   Body=payload)
        predictions = ','.join([predictions, response['Body'].read().decode('utf-8')])
        time.sleep(0.5)

predictions = predictions.replace('\n','')
predictions = predictions.split(",")
predictions.pop(0)
print("="*20)
print(predictions)
print("Done!") 

## キャプチャしたデータを表示する

以下のコードを実行し、Amazon S3 に保存されたデータキャプチャファイルを一覧表示します。さまざまな時間枠のさまざまなファイルが、呼び出しが発生した時間に基づいて並べられます。Amazon S3 パスのフォーマットは以下のとおりです。

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [ ]:
print("Waiting 2 minutes for file to load...")
time.sleep(120)
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
print("Found Capture Files:")
print("\ns3://"+bucket+str(capture_files[0]))

次に、あるキャプチャファイルの内容を表示します。すべてのデータが、Amazon SageMaker 固有の JSON 行形式ファイルにキャプチャされているはずです。キャプチャされたファイルの最初の数行を見てみましょう。

In [ ]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

最後に、ある行の内容が以下のとおり、フォーマット済みの JSON ファイルに表示され、少しわかりやすくなりました。

In [ ]:
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

## Amazon Sagemaker Clarify のセットアップ


**注意:** 以下のコードサンプルは、Amazon SageMaker の[サンプルコード](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_processing/fairness_and_explainability/fairness_and_explainability.ipynb)から抽出されました。コードは一部がラボ要件に合わせて更新されています。

SageMaker Clarify processor を使用するため以下のコードを実行します。
Run the following code to use the SageMaker Clarify processor.

Clarify processor は分散クラスター上でバイアスの検出に利用するインフラストラクチャをセットアップします。この例では、クラスター上で 1 インスタンスのみです。より詳しく Clarify processor の設定について知りたい場合は、こちらを参照します [Configure an Amazon SageMaker Clarify Processing Jobs for Fairness and Explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-configure-processing-jobs.html)。

In [ ]:
from sagemaker import clarify
clarify_processor = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.m5.xlarge',
                                                      sagemaker_session=sagemaker_session)

テストを S3 にアップロードしてデータセットをトレーニングします。_train_uri_ および _test_uri_ は、次のステップで dataconfig および modelconfig を書き込む S3 からデータを直接プルするために使用されます。

In [ ]:
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

train_uri = S3Uploader.upload('data/train.csv', 's3://{}/{}'.format(bucket, prefix))
train_input = TrainingInput(train_uri, content_type='csv')
test_uri = S3Uploader.upload('data/test.csv', 's3://{}/{}'.format(bucket, prefix))

## バイアスを検出する

#### SageMaker Clarify はさまざまなメトリクスを使用して、トレーニングの前後に存在しうるバイアスを検出できるように支援します。

### DataConfig および ModelConfig を書き込む

**DataConfig** オブジェクトは、データ I/O に関するベーシックな情報のいくつかを SageMaker Clarify に伝達します。以下のとおり、入力データセットを探す場所、出力を保存する場所、対象列 (ラベル)、ヘッダー名、データセットタイプを指定します。

以下のコードを実行し、DataConfig を定義します。

In [ ]:
bias_report_output_path = 's3://{}/{}/clarify-bias'.format(bucket, prefix)
bias_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                      s3_output_path=bias_report_output_path,
                                      label='Churn',
                                      headers=all_headers,
                                      dataset_type='text/csv')

**ModelConfig** オブジェクトはトレーニング済みのモデルに関する情報を伝達します。本番稼動モデルへのトラフィックが増えないように、処理中は SageMaker Clarify が専用エンドポイントをセットアップしたり、削除したりします。

- **instance_type** および **instance_count** は、SageMaker Clarify の処理中にモデル実行に使用したいインスタンスタイプとインスタンス数を指定する。テスト用データセットは小さいため、この例を実行するにはスタンダードインスタンスが 1 個あれば十分である。大規模で複雑なデータセットであれば、より上位のインスタンスタイプを使用して高速化できる。Spark 並列化を有効にしたいのであれば、インスタンス数を追加する

- **accept_type** は、エンドポイントのレスポンスペイロードのフォーマットを表す。**content_type** は、エンドポイントに対するリクエストのペイロードフォーマットを表す

以下のコードを実行し、ModelConfig を定義します。

In [ ]:
model_config = clarify.ModelConfig(model_name=model_name,
                                   instance_type='ml.m5.xlarge',
                                   instance_count=1,
                                   accept_type='text/csv',
                                   content_type='text/csv')

**ModelPredictedLabelConfig** では、予測のフォーマットに関する情報を得られます。ランダムフォレストモデルによってサンプルの確率が出力されると、SageMaker Clarify がエンドポイントを呼び出し、probability_threshold を使用して、その確率をバイアス分析のバイナリラベルに変換します。このしきい値を上回る予測はラベル値 1、しきい値と同じか下回る予測はラベル値 0 として解釈されます。

以下のコードを実行し、ModelPredictedLabelConfig を定義します。

In [ ]:
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

### BiasConfig を書き込む
SageMaker Clarify には、センシティブな列 (facets)、センシティブな特徴 (facet_values_or_threshold)、望ましい結果 (label_values_or_threshold) に関する情報も必要です。SageMaker Clarify は、facet_values_or_threshold および label_values_or_threshold に対して、カテゴリ型データと連続型データの両方を処理できます。このケースでは、カテゴリ型データを使用しています。

この情報を BiasConfig API で指定します。前向きな結果として、Churn = 0、_Account Length_ がセンシティブなカテゴリ、_Day Calls_ の応答者がセンシティブなグループとなりました。group_name は計測対象のサブグループの形成に使用されます。

In [ ]:
bias_config = clarify.BiasConfig(label_values_or_threshold=[0],
                                facet_name='Account Length',
                                facet_values_or_threshold=[100],
                                group_name='Day Calls')

#### **トレーニング前のバイアス**

バイアスは、モデルトレーニングの開始前にデータに存在していることがあります。トレーニング前にデータのバイアスの有無を点検しておくと、データ収集の際のギャップ検出、特徴エンジニアリングの伝達、データが内包する社会的バイアスの理解に役立ちます。

トレーニング前のバイアスメトリクスのコンピューティングには、トレーニング済みのモデルは必要ありません。

#### **トレーニング後のバイアス**

トレーニング後のバイアスメトリクスのコンピューティングには、トレーニング済みのモデルが必須となります。

バイアスのないトレーニングデータ (バイアスメトリクスで計測された公平性の概念で決まる) であっても、トレーニング後のモデル予測にバイアスが生じることがあります。この現象には、ハイパーパラメータの選択といったいくつかの要因があります。

これらのオプションを、run_pre_training_bias() と run_post_training_bias() で別個に実行するか、または以下のように run_bias() で同時に実行します。

In [ ]:
clarify_processor.run_bias(data_config=bias_data_config,
                           bias_config=bias_config,
                           model_config=model_config,
                           model_predicted_label_config=predictions_config,
                           pre_training_methods='all',
                           post_training_methods='all')


#### **バイアスレポートを表示する**

SageMaker Studio では、結果は実験タブに表示されます。

1. 左ペインで, **SageMaker resources** をクリック
1. ドロップダウンメニューから **Experimentals and trials** を選択
1. **Unassigned trial components** を開く (ダブルクリック)
1. 名前に **clarify-bias** を含む **trial** を開く (ダブルクリック)

  新規タブにバイアスレポートが表示されます。内容を確認後、残りの手順を実行するためこのタブに戻ります。

1. 開いた新規タブで **bias report** を選択

<img src="./recordings/bias_report.gif">

各バイアスメトリクスは、詳細に確認できるサンプル付きで詳しく説明されています。

- **Class Imbalance(CI):** 有利なグループが不利なグループよりも大幅に高い割合でデータセットに表示されるかどうか、またはその逆かどうかを検出します。
- **Total Variation Distance (TVD):** この値は有利なクラスのラベルの確率分布と不利なクラスの確率分布の間のハミング距離の半分です。
- **Conditional Demographic Disparity in Predicted Labels (CDDPL):** モデルの予測が、不利な立場にあるクラスに対し受け入れられた結果の割合よりも、拒否された結果の割合が大きいかどうかを調べます

<img src="./recordings/bias_detail.gif">

結果は便利な表にまとめることもできます。

<img src="./recordings/bias_report_chart.gif">



SageMaker Studio のユーザーでない場合は、以下の S3 バケットから pdf、html、ipynb 形式のバイアスレポートにアクセスできます。

In [ ]:
bias_report_output_path

## 予測を説明する

企業や規制当局においては、あるモデルがその決定を下した理由の説明を求める動きが高まりつつあります。SageMaker Clarify では、SHAP を使用して、最終決定に対する各入力の特徴の寄与が説明されます。

Kernel SHAP アルゴリズムにはベースライン (バックグラウンドデータセットとも呼ばれる) が必要です。ベースラインデータセットタイプは DataConfig の dataset_type と同じであり、ベースラインのサンプルには特徴だけが含まれます。定義によれば、ベースラインはベースラインデータセットファイルの S3 URI か、サンプルのインプレースリストのいずれかである必要があります。このケースでは後者を選び、テストデータセットの最初のサンプルをリストに載せます。

In [ ]:
shap_config = clarify.SHAPConfig(baseline=[test_data.iloc[0].values.tolist()],
                                 num_samples=15,
                                 agg_method='mean_abs',
                                 save_local_shap_values=False)

explainability_output_path = 's3://{}/{}/clarify-explainability'.format(bucket, prefix)
explainability_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                s3_output_path=explainability_output_path,
                                label='Churn',
                                headers=all_headers,
                                dataset_type='text/csv')

セルを実行しClarify を開始し説明性の評価をします。タスクを完了するには通常 10 分程度かかります。

プロセスが完了するまでの間、こちらのリンク [Clarify fairness and explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-fairness-and-explainability.html) で詳細を確認することが可能です 。

In [ ]:
clarify_processor.run_explainability(data_config=explainability_data_config,
                                     model_config=model_config,
                                     explainability_config=shap_config)

##### 説明可能性レポートを表示する
バイアスレポートと同様に、SageMaker Studio の実験タブで説明可能性レポートを表示できます。

1. 左ペインで, **SageMaker resources** をクリック
1. ドロップダウンメニューから **Experimentals and trials** を選択
1. **Unassigned trial components** を選択
1. **cclarify-explainabilit** が名前に含まれたものを選択し、 **Explainability** タブを選択しレポートを確認

<img src="./recordings/explainability_detail.gif">

#### **問題:** レポートによると、最も解約予測に重要なのはどの特徴量でしょう？
[**Model Insights**] タブに、レポートとモデルインサイトへの直接リンクがあります。

SageMaker Studio のユーザーでない場合は、バイアスレポートと同様に以下の S3 バケットから説明可能性レポートにアクセスできます。

In [ ]:
explainability_output_path

### クリーンアップ

最後に、このデモでセットアップおよび使用したリソースは必ずクリーンアップしておいてください。

In [ ]:
sagemaker_session.delete_model(model_name)